<a href="https://colab.research.google.com/github/bhushanmandava/basic_transformer_architecture/blob/main/llmImpplemntation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class attentionHead(nn.Module):
  def __init__(self,emb_dim,d_h):
    super().__init__()
    self.w_q = nn.Parameter(torch.empty(emb_dim,d_h))
    self.w_k = nn.Parameter(torch.empty(emb_dim,d_h))
    self.w_v = nn.Parameter(torch.empty(emb_dim,d_h))
    self.d_h= d_h
  def forward(self,x,mask):
    q = x @self.w_q
    k= x @ self.w_k
    v= x @ self.w_v
    q , k =rope(q),rope(k)
    score = q @ k.transpose(-2,-1) / math.sqrt(self.d_h)
    masked_scores = score.masked_fill(mask == 0 , float("-inf"))
    attention_weights = torch.softmax(masked_scores,dim=-1)
    return attention_weights @ v
class MultiHeadAttention(nn.Module):
  def __init__(self, emb_dim, number_heads):
    super().__init__()
    self.d_h = emb_dim // number_heads
    self.heads =nn.ModuleList([
        attentionHead(self.emb_dim,self.d_h)for _ in range(number_heads)
    ])
    self.w_o = nn.parameter(torch.empty(emb_dim,emb_dim))
    def forward(self,x,mask):
      output = torch.cat([head(x,mask) for head in self.heads])
      return output @ self.w_0
class MLP(nn.Module):
  def __init__(self,emb_dim):
    super().__init__()
    self.w_1 = nn.paramter(torch.empty(emb_dim,4*emb_dim))
    self.w_2 = nn.paramter(torch.empty(4*emb_dim,emb_dim))
    self.b_1 = nn.paramter(torch.empty(4*emb_dim))
    self.b_2 = nn.paramter(torch.empty(emb_dim))
    def forward(self, x):
      x = x @ self.w_1 + self.b_1
      x = torch.relu(x)
      x = x @ self.w_2 + self.b_2
      return x
class TransformerModule(nn.Module):
  def __init__(self,emb_dim,number_heads):
    super().__init__()
    self.norm1 = nn.RMSNorm(emb_dim)
    self.attn= MultiHeadAttention(emb_dim,number_heads)
    self.norm2 = nn.RMSNorm(emb_dim)
    self.mlp = MLP(emb_dim)
  def forward(self,x,mask):
    x = x+self.attn(self.norm1(x),mask)
    x = x+self.mlp(self.norm2(x))
    return x

class autoregressiveTransformer(nn.Module):
  def __init__(self, vocab_size , emb_dim , number_heads ,number_blocks,pad_idx):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size , emb_dim , padding_idx=pad_idx)
    self.layers = nn.ModuleList( [
        TransformerModule(emb_dim,number_heads) for _ in range(number_blocks) ])
    self.out = nn.Parameter(torch.rand(emb_dim,vocab_size))
  def forward(self,x,mask):
    x = self.embedding(x)
    _,seq_len,_ =x.shape
    mask = torch.tril(torch.ones(seq_len,seq_len,device = x.device))
    for layer in self.layers:
      x = layer(x,mask)
    logits = x @ self.out
    return logits